# Задания

In [1]:
import numpy as np
import pandas as pd

## Задание 1

Загрузи данные о посещениях заведений. В них содержится информация о регистрации пользователя в заведениях 
и геолокация этих заведений. Очисти данные от записей с пропусками. \
Выведи количество записей после очистки. 

In [2]:
# Считываем файл с инфой о посещениях как таблицу, разделитль черточка, удаляем все лишние начальные пробелы, для строк, где нет значений широты и долготы меняем параметр на Nan
df = pd.read_table('../datasets/checkins.dat', sep = '|', low_memory = False, skipinitialspace=True, na_values = '')

In [3]:
### Проверяем заголовки столбцов таблиц
df.columns

Index(['id    ', 'user_id ', 'venue_id ', 'latitude      ', 'longitude     ',
       'created_at      '],
      dtype='object')

In [4]:
### Удаляем лишние пробелы из названий столбцов таблицы (после названия)
df.columns = [name.strip() for name in df.columns]
df.columns

Index(['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at'], dtype='object')

In [5]:
### Посморим, как выглядит таблица. Видим, что есть лишняя строка и значения координат NaN
df.head

<bound method NDFrame.head of                                                         id    user_id   
0        ---------+---------+----------+---------------...        NaN  \
1                                                  984301   2041916.0   
2                                                  984222     15824.0   
3                                                  984315   1764391.0   
4                                                  984234     44652.0   
...                                                    ...        ...   
1021963                                            956119   1139114.0   
1021964                                            956447   2088020.0   
1021965                                            956733    960666.0   
1021966                                            957139   1771518.0   
1021967                                     (1021966 rows)        NaN   

         venue_id   latitude  longitude           created_at  
0             NaN        NaN  

In [6]:
df.dropna(inplace=True)
df.head

<bound method NDFrame.head of               id    user_id  venue_id   latitude   longitude   
2        984222     15824.0    5222.0  38.895112  -77.036366  \
4        984234     44652.0    5222.0  33.800745  -84.410520   
8        984291    105054.0    5222.0  45.523452 -122.676207   
10       984318   2146539.0    5222.0  40.764462 -111.904565   
11       984232     93870.0  380645.0  33.448377 -112.074037   
...          ...        ...       ...        ...         ...   
1021960  955561    626076.0   20073.0  40.850100  -73.866246   
1021961  955892    674797.0    2297.0  33.748995  -84.387982   
1021962  956377    845102.0   11195.0  42.765366  -71.467566   
1021963  956119   1139114.0   29488.0  42.439479  -83.743830   
1021965  956733    960666.0      60.0  42.331427  -83.045754   

                  created_at  
2        2012-04-21 17:43:47  
4        2012-04-21 17:43:43  
8        2012-04-21 17:39:22  
10       2012-04-21 17:35:46  
11       2012-04-21 17:38:18  
...            

In [7]:
print ('Количество записей после очистки = ', df.shape[0])

Количество записей после очистки =  396634


## Задание 2

Эти данные содержат информацию о заведениях со всего мира. С помощью геолокаций и библиотеки [Reverse Geocoder](https://github.com/thampiman/reverse-geocoder),
узнай страну для каждой геопозиции. \
Узнай **название** второй страны по количеству записей.

In [8]:
# Код тут
import reverse_geocoder as rg
import gettext
import pycountry

## Функция, которая по коду страны возвращает название стран и переводит его на руусский язык
def get_country(code):
    ru = pycountry.countries.get(alpha_2=code)
    if not ru is None:
        return russian.gettext(ru.name)


## Подклчаем русский язык
russian = gettext.translation(
    'iso3166', pycountry.LOCALES_DIR, languages = ['ru'])
russian.install

## обрабаттываем датафрейм, на каждое значение возвращаем название страны, делаем подсчет с сортировкой по убыванию и выводим вторую страну
df['country'] = [get_country(y.get('cc')) for y in rg.search(
        [tuple(x) for x in df[['latitude', 'longitude']].to_numpy()])]
#df['country'].value_counts()

print('Вторая страна по количеству записей - это', df['country'].value_counts().index[1])


Loading formatted geocoded file...
Вторая страна по количеству записей - это Индонезия


## Задание 3

Нас будут интересовать только американские локации. Очисти данные от локаций, находящихся в других странах. 
Также, чтобы сократить количество, геолокаций оставь в выборке только 50 самых часто встречаемых заведений (venue). \
Выведи количество локаций, оставшихся после этих очисток.

In [9]:
## Делаем отбор по странеи США
df1 = df[df.country == 'Соединённые штаты'].copy()

## Сохраняю выборку из 50 самых часто встречаемых локаций
top50 = df1.venue_id.value_counts().index[:50].to_list()

## Считаем количество локаций, оставшихся после чисток
df1.query('venue_id in @top50', inplace = True)
print('Количество оставшихся локаций:', df1.shape[0])

Количество оставшихся локаций: 162099


## Задание 4

Перейдем к задаче кластеризации. Воспользуйся алгоритмом [Mean Shift](https://scikit-learn.org/stable/modules/clustering.html#mean-shift)
для кластеризации локаций. Параметрами укажи `MeanShift(bandwidth=0.1, bin_seeding=True)`. 

    `bandwidth=0.1` - это ширина ядра кластеризации. Для средних широт США - это порядка 5-10 км. 
    `bin_seeding=True` - для ускорения работы алгоритма.
    
Выведи количество кластеров, которые у тебя получились в результате кластеризации.

In [10]:
# Код тут
from sklearn.cluster import MeanShift
ms = MeanShift(bandwidth=0.1, bin_seeding=True).fit(
    df1[['latitude', 'longitude']])
print('Количество кластеров = ', len(ms.cluster_centers_))

Количество кластеров =  2846


## Задание 5

Центры полученных кластеров - это потенциальные места установки банеров компании. Теперь хотелось бы найти те центры кластеров, 
которые наиболее близко находятся к офисам продаж компании. \
Загрузи [данные по координатам офисов компании](datasets/offices.csv). Для каждого офиса найди 5 самых ближайших к нему центров кластеров. 
(Пренебрежем тем, что Земля круглая и рассчитаем Евклидово расстояние).
У компании 11 офисов, значит у нас должно получится 55 мест установки баннеров. \
Выведи координаты установки баннера, который ближе всего находится к офису компании.

In [11]:
# Бинарный поиск (артеллериская вилка) в 8 раз быстрее. Поиск в виде KDTree - трехмерное бинарное дерево. Каждые две координаты переводим в три координаты и осуществяем трехмерный поиск, он быстрее

import scipy.spatial as spatial
from math import radians, cos, sin, asin, sqrt

R = 6367 # радиус Земли в км в одном радиане

## Создаем датарейм, который содержит наши 50 кластеров из прошлого задания
df_centers = pd.DataFrame(ms.cluster_centers_.tolist(),
                          columns=['latitude', 'longitude'])

##Перводим координаты широты и долготы в новые трехмерные координаты
phi = np.deg2rad(df_centers['latitude'])
theta = np.deg2rad(df_centers['longitude'])
df_centers['x'] = R * np.cos(phi)*np.cos(theta)
df_centers['y'] = R * np.cos(phi)*np.sin(theta)
df_centers['z'] = R * np.cos(phi)


df_offices = pd.read_csv('../datasets/offices.csv', index_col=0)
phi = np.deg2rad(df_offices['latitude'])
theta = np.deg2rad(df_offices['longitude'])
df_offices['x'] = R * np.cos(phi)*np.cos(theta)
df_offices['y'] = R * np.cos(phi)*np.sin(theta)
df_offices['z'] = R * np.cos(phi)

##Создаем дерево и делаем к нему запросы на олучение пяти ближайших точек из дерева
tree = spatial.KDTree(df_centers[['x', 'y', 'z']])
distance, index = tree.query(df_offices[['x', 'y', 'z']], k=5)
nearest = np.unravel_index(distance.argmin(), distance.shape)[0] + 1
lat_nearest, lon_nearest = df_centers[['latitude', 'longitude']
                                      ].loc[index[np.unravel_index(distance.argmin(), distance.shape)]]
print('Ближе всего к офису № {} находится баннер с координатами:\nlat:{}, lot:{}'.
      format(nearest, lat_nearest, lon_nearest))


Ближе всего к офису № 10 находится баннер с координатами:
lat:32.78531777579914, lot:-79.92474241187222


## Задание 6

С помощью функции [scatter_mapbox](https://plotly.github.io/plotly.py-docs/generated/plotly.express.scatter_mapbox.html) 
отметь точки установки баннеров. У тебя должна получится такая картинка.
Цветом точки укажи к какому офису будет относиться этот баннер. 
>Цвет легенды может отличаться от референса

<center><img src="../misc/images/task_6.png" width="800" height="800"/> <center/>

In [12]:
# Код тут
from plotly.express import scatter_mapbox
df_banners = df_centers.iloc[np.ravel(index).tolist()].copy()
df_banners['office'] = [str(i) for i in range(
    len(index)) for _ in range(len(index[0]))]
fig = scatter_mapbox(df_banners, lat=df_banners.latitude, lon=df_banners.longitude,
                     color = df_banners.office, mapbox_style='open-street-map',
                     zoom = 2.9, width=800, height = 800, )
fig.show()